In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

aug_train_data = ImageDataGenerator(rescale=1./255, 
                                shear_range=0.3,
                                zoom_range=0.2,
                                rotation_range=40,
                                width_shift_range=0.25,
                                height_shift_range=0.25,
                                fill_mode='nearest')

aug_valid_data = ImageDataGenerator(rescale=1./255)

batch_size = 32
base_dir = "../input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)"

train_data = aug_train_data.flow_from_directory(base_dir + "/train",
                                                target_size=(224, 224),             #Default InceptionV3
                                                batch_size=batch_size,
                                                class_mode='categorical')

valid_data = aug_valid_data.flow_from_directory(base_dir + "/valid",
                                                target_size=(224, 224),
                                                batch_size=batch_size,
                                                class_mode='categorical')

In [ ]:
classes = list(train_data.class_indices.keys())
print(*classes, sep='\n')

In [ ]:
train_num = train_data.samples
valid_num = valid_data.samples
print(f"Total output classes: {len(classes)} \nTotal Training samples: {train_num} \nTotal Validation samples: {valid_num}")

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

In [ ]:
from keras.models import Sequential,load_model,Model
from keras.applications import InceptionV3
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_v3 import preprocess_input

# base_model = load_model('../input/modelarch/base_model.h5')
base_model = InceptionV3(weights='imagenet', include_top=False)

CLASSES = 38

x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)

predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer="RMSprop",
              loss='categorical_crossentropy',
              metrics=["accuracy"])

In [ ]:
EPOCHS = 25
BATCH_SIZE = 128
STEPS_PER_EPOCH = 550
VALIDATION_STEPS = 64
savepath = "inceptionV3_rmsprop_augmented.h5"

history = model.fit_generator(
    train_data,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=valid_data,
    validation_steps=VALIDATION_STEPS)

model.save(savepath)

In [ ]:
import seaborn as sns
sns.set()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
# predicting an image
from keras.preprocessing import image
import numpy as np

def predictor(model, image_path):

    new_img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(new_img)
    img = np.expand_dims(img, axis=0)
    img = img/255

    print("True Label:", image_path.replace("../input/new-plant-diseases-dataset/test/test/", ""))
    print("Following is our prediction:")
    prediction = model.predict(img)

    decoded = prediction.flatten()
    j = decoded.max()
    for index,item in enumerate(decoded):
        if item == j:
            class_name = classes[index]

    #ploting image with predicted class name        
    plt.figure(figsize = (4,4))
    plt.imshow(new_img)
    plt.axis('off')
    plt.title(class_name)
    plt.show()

image_path = "../input/new-plant-diseases-dataset/test/test/TomatoHealthy3.JPG"
predictor(model, image_path)